1. pair representations $h_v^{(0)}$  赋值子图中每个节点和源节点信息
2. Message $M_v^{(1)}$: {$h_v^{(0)}$, 所有指向v的节点x的$h_x$和(x, v)总结后的信息}
3. 总结所有Message

In [1]:
import pandas as pd
sample = pd.read_excel('sample.xlsx')
cite = pd.read_csv('sample_cite.csv')

# 第一步
src = sample[sample.pid == 35]
# tar = sample[sample.pid == 63486]
v_df = sample[sample.pid != src.pid.values[0]].reset_index(drop=True)

h_v = pd.DataFrame({
    'src_pid': [src.pid.values[0]] * len(v_df),
    'src_title': [src.title.values[0]] * len(v_df),
    'src_abs': [src['abs'].values[0]] * len(v_df),
    'src_label': [src.label.values[0]] * len(v_df),
    'v_pid': v_df.pid,
    'v_title': v_df.title,
    'v_abs': v_df['abs'],
    'v_label': v_df.label
})

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa1 in position 6503: invalid start byte

In [13]:
sample

,pid,title,abs,label,keywords,h0
0,32083,Title: Back Propagation is Sensitive to Initia...,Abstract: This paper explores the effect of in...,Neural_Networks,"Back propagation, initial weight selection, fe...",Title: 4 Implementing Application Specific Rou...
1,273152,Title: Evolutionary Module Acquisition,Abstract: A critical issue for users of Markov...,Genetic_Algorithms,"convergence diagnostics, convergence assessmen...",Title: 4 Implementing Application Specific Rou...
2,35061,Title: Evolving Networks: Using the Genetic Al...,Abstract: A pilot study is described on the pr...,Genetic_Algorithms,"Artificial neural networks, attitude control, ...",Title: 4 Implementing Application Specific Rou...
3,44514,Title: The Observers Paradox: Apparent Computa...,Abstract: Many connectionist approaches to mus...,Neural_Networks,"Connectionist approaches, musical expectancy, ...",Title: 4 Implementing Application Specific Rou...
4,35,Title: 4 Implementing Application Specific Rou...,"Abstract: To implement a specific application,...",Genetic_Algorithms,"Genetic algorithms, application-specific routi...",NaN
5,210871,Title: Coevolving High-Level Representations,Abstract: Rules extracted from trained feedfor...,Genetic_Algorithms,"Rule extraction, feedforward networks, perform...",Title: 4 Implementing Application Specific Rou...
6,210872,Title: An Evolutionary Algorithm that Construc...,Abstract: Standard methods for inducing both t...,Genetic_Algorithms,"Recurrent neural networks, genetic algorithms,...",Title: 4 Implementing Application Specific Rou...
7,82920,Title: A Survey of Evolution Strategies,Abstract:,Genetic_Algorithms,"Evolution strategies, optimization, machine le...",Title: 4 Implementing Application Specific Rou...
8,141342,Title: Dynamic Parameter Encoding for Genetic ...,Abstract: The common use of static binary plac...,Genetic_Algorithms,"Genetic algorithms, Dynamic Parameter Encoding...",Title: 4 Implementing Application Specific Rou...


In [14]:
# Indicator
# h0

for i in range(len(h_v)):

    src_title, src_abs, src_label = h_v.loc[i, 'src_title'], h_v.loc[i, 'src_abs'], h_v.loc[i, 'src_label']
    tar_pid, tar_title, tar_abs, tar_label = h_v.loc[i, 'v_pid'], h_v.loc[i, 'v_title'], h_v.loc[i, 'v_abs'], h_v.loc[i, 'v_label']

    # if tar['pid'].values[0] == tar_pid:
    #     continue
    
    print(tar_pid)

    Indicator_prompt = f"""
Given information of a source paper and a target paper shown as follow:
source paper:
title:{src_title}
abstract:{src_abs}
label:{src_label}
target paper:
title:{tar_title}
abstract:{tar_abs}
label:{tar_label}
please summarize information of these two papers in no more than three sentences, and then generate some keywords of this information.
Answer format:
\*
Summarization: [your summarization]
Keywords:
{src_title}: 
[source paper keywords]
{tar_title}:
[target paper keywords]
\*
When outputting your answer, when referring to a paper, you should just state the title of the paper, don't mention the source paper or targer paper.
"""

    print(Indicator_prompt)
    print('======================================================================================')


32083

Given information of a source paper and a target paper shown as follow:
source paper:
title:Title: 4 Implementing Application Specific Routines  Genetic algorithms in search, optimization, and machine learning. Reading, MA: Addison-Wesley.
abstract:Abstract: To implement a specific application, you should only have to change the file app.c. Section 2 describes the routines in app.c in detail. If you use additional variables for your specific problem, the easiest method of making them available to other program units is to declare them in sga.h and external.h. However, take care that you do not redeclare existing variables. Two example applications files are included in the SGA-C distribution. The file app1.c performs the simple example problem included with the Pascal version; finding the maximum of x 10 , where x is an integer interpretation of a chromosome. A slightly more complex application is include in app2.c. This application illustrates two features that have been added 

In [4]:
# message
# 遍历cite表所有src_id 不是u的边，获得对应的src_id(x)
# 从sample表里获得x的h0，并和cite关系w聚合，得到m
# 将m存入cite表

for i in range(len(cite)):
    src_id, tar_id = cite.loc[i, 'src_id'], cite.loc[i, 'tar_id']
    
    # 隐去所有以src_id为源的边
    if src_id == src.pid.values[0]:
        # print(src_id, tar_id)
        continue
    
    src_title, src_abs, src_label = sample[sample.pid == src_id].title.values[0], sample[sample.pid == src_id]['abs'].values[0], sample[sample.pid == src_id].label.values[0]
    tar_title, tar_abs, tar_label = sample[sample.pid == tar_id].title.values[0], sample[sample.pid == tar_id]['abs'].values[0], sample[sample.pid == tar_id].label.values[0]
    
    # hx
    hx = sample[sample.pid == src_id][f'h0_{src.pid.values[0]}'].values[0]

    w_prompt = f"""title:{src_title}, label:{src_label} cites title:{tar_title}, label:{tar_label}."""

    message_prompt = f"""
Given the source and target papers, and information about the paper cited by target paper:
information of source and target papers: {hx}

the paper which target paper cites: {w_prompt}

please summarize information of source and target papers and the citation relationship in no more than three sentences, and then generate some keywords of this information.
When outputting your answer, when referring to a paper, you should state the title of the paper and indicate whether it is a source paper or a target paper.
"""

    print(src_id, tar_id)
    
    print(message_prompt)

    print('================================================================================')

2440 72908

Given the source and target papers, and information about the paper cited by target paper:
information of source and target papers: The source paper, "Learning Boolean Concepts in the Presence of Many Irrelevant Features," addresses case-based learning amid irrelevant features, proposing the Oblivion algorithm for feature identification, particularly in interacting scenarios like parity concepts. It discusses experimental results and implications for future research. The target paper, "Quinlan, 1990 J.R. Quinlan. Learning logical definitions from relations," introduces a ranked-model semantics for if-then rules, facilitating evidential and causal reasoning by extracting rule priorities automatically. It resolves problems related to specificity, prediction, abduction, and unifies belief revision, update, and reasoning about actions.

Keywords:
Source paper: Case-based learning, Irrelevant features, Oblivion algorithm, Feature identification, Parity concepts, Experimental res

In [5]:
# 遍历每个不是源点的sample
# 找到所有以该点为终点的边，组合message
# 

src_id = src.pid.values[0]

for i in range(len(sample)):
    v_id, h0 = sample.loc[i, 'pid'], sample.loc[i, f'h0_{src_id}']
    m_list = cite[(cite.tar_id == v_id) & (cite.src_id != src_id)][f'm_{src_id}'].values

    # print(v_id, m_list)

    agg_prompt = f'''
given information of a source paper and a target paper shown as follow:
{h0}
'''

    print(v_id)
    if len(m_list) > 0:
        agg_prompt = f'''{agg_prompt}\n
the information of paper which cites the target paper and their relation of the source paper shown as follow:
'''
        for j, m in enumerate(m_list):
            agg_prompt = f"{agg_prompt}\n{j+1}. [{m}]\n"
    agg_prompt = f'''{agg_prompt}
please summarize information in no more than three sentences, and then generate some keywords of this information.
When outputting your answer, when referring to a paper, you should state the title of the paper and indicate whether it is a source paper or a target paper.
'''
    print(agg_prompt)
    print('================================================================================================')


63486

given information of a source paper and a target paper shown as follow:
The source paper, "Learning Boolean Concepts in the Presence of Many Irrelevant Features," explores addressing case-based learning amidst irrelevant features, proposing the Oblivion algorithm for efficient feature identification, especially with interacting features like parity concepts. It discusses experimental results in artificial domains supporting its hypothesis, noting varied outcomes in natural domains, concluding with considerations for future research. The target paper, "Evaluation and Selection of Biases in Machine Learning," introduces and defines biases in machine learning, emphasizing the significance of automated methods for bias evaluation and selection through frameworks of bias/meta-bias spaces, summarizing recent research in this area.

Keywords:
Source paper: Case-based learning, Irrelevant features, Oblivion algorithm, Feature identification, Parity concepts, Experimental results
Target 

In [6]:
# prediction

src_title = sample[sample.pid == src_id]['title'].values[0]


for i in range(len(sample)):
    if sample.loc[i, 'pid'] == src_id:
        continue

    tar_pid, tar_title = sample.loc[i, 'pid'], sample.loc[i, 'title']

    h1 = sample.loc[i, f'h1_{src_id}']
    print(src_id, tar_pid)


    prediction_prompt = f"""
given information of a source paper and a target paper shown as follow:
{h1}
Is there an existing citation relationship between "{src_title}" and "{tar_title}"
Do not give any reasoning or logic for your answer. Answer Yes if they have cited relationship, otherwise, answer No.
answer:

"""
    print(prediction_prompt)
    print('=========================')

10169 63486

given information of a source paper and a target paper shown as follow:
The source paper, "Learning Boolean Concepts in the Presence of Many Irrelevant Features," introduces the Oblivion algorithm for case-based learning amidst irrelevant features, emphasizing feature identification, particularly in scenarios involving interacting concepts like parity. It explores experimental findings and future research directions on irrelevant features. In one cited target paper, "Irrelevant Features and the Subset Selection Problem," the focus shifts to identifying relevant feature subsets for high-accuracy concepts in supervised induction algorithms through refined relevance concepts and cross-validation methods, citing the theoretical aspects of biases in machine learning from the target paper, "Evaluation and Selection of Biases in Machine Learning."

Keywords:
Source paper: Case-based learning, Irrelevant features, Oblivion algorithm, Feature identification, Parity concepts, Experi

In [21]:
# g
# 总结 h0 & (h0, q)

src_id, src_title = src.pid.values[0], src.title.values[0]
src_keywords = sample[sample.pid == src_id].keywords

for i in range(len(sample)):

    tar_id, tar_title, h0, tar_keywords = sample.loc[i, 'pid'], sample.loc[i, 'title'], sample.loc[i, 'h0'], sample.loc[i, 'keywords']

    if tar_id == src_id:
        continue

    print(tar_id)
    g_prompt = f"""
Given information of two paper shown as follow:
paper1: {src_title}
paper2: {tar_title}
information: {h0}
{src_title} keywords: {src_keywords}
{tar_title} keywords: {tar_keywords}

please summarize information of these two papers in no more than three sentences and assess the likelihood that they have references to each other.
Answer format:
\
Summarization: [your summarization]
Citation possibility: [your assessment]
\
When outputting your answer, when referring to a paper, you should just state the title of the paper, don't mention paper1 or paper2, the first paper or the second paper, just say paper title.
"""
    print(g_prompt)
    print('=========================')

32083

Given information of two paper shown as follow:
paper1: Title: 4 Implementing Application Specific Routines  Genetic algorithms in search, optimization, and machine learning. Reading, MA: Addison-Wesley.
paper2: Title: Back Propagation is Sensitive to Initial Conditions
information: Title: 4 Implementing Application Specific Routines Genetic algorithms in search, optimization, and machine learning. Reading, MA: Addison-Wesley.: Discusses implementing application-specific routines in genetic algorithms, emphasizing changes in the 'app.c' file, utilizing 'sga.h' and 'external.h' for variable declaration, and offering examples of applications within the SGA-C distribution.
Title: Back Propagation is Sensitive to Initial Conditions: Explores the impact of initial weight selection on feed-forward networks using the back-propagation technique, highlighting through Monte Carlo and deterministic experiments the significant influence of initial weight configuration on convergence time va